In [ ]:
!pip install -U skorch

In [ ]:
import numpy as np
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
from skorch import NeuralNetRegressor

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
test  = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')

In [ ]:
train=pd.concat([train,pd.get_dummies(train.cat0,prefix='cat0')],axis=1)
train=train.drop(['cat0','cat0_B'],axis=1)
train=pd.concat([train,pd.get_dummies(train.cat1,prefix='cat1')],axis=1)
train=train.drop(['cat1','cat1_B'],axis=1)
train=pd.concat([train,pd.get_dummies(train.cat2,prefix='cat2')],axis=1)
train=train.drop(['cat2','cat2_B'],axis=1)
train=pd.concat([train,pd.get_dummies(train.cat3,prefix='cat3')],axis=1)
train=train.drop(['cat3','cat3_D'],axis=1)
train=pd.concat([train,pd.get_dummies(train.cat4,prefix='cat4')],axis=1)
train=train.drop(['cat4','cat4_D'],axis=1)
train=pd.concat([train,pd.get_dummies(train.cat5,prefix='cat5')],axis=1)
train=train.drop(['cat5','cat5_D'],axis=1)
train=pd.concat([train,pd.get_dummies(train.cat6,prefix='cat6')],axis=1)
#df_test=df_test.drop(['cat6'],axis=1)
train=train.drop(['cat6','cat6_I'],axis=1)
train=pd.concat([train,pd.get_dummies(train.cat7,prefix='cat7')],axis=1)
train=train.drop(['cat7','cat7_I'],axis=1)
train=pd.concat([train,pd.get_dummies(train.cat8,prefix='cat8')],axis=1)
train=train.drop(['cat8','cat8_G'],axis=1)
train=pd.concat([train,pd.get_dummies(train.cat9,prefix='cat9')],axis=1)
train=train.drop(['cat9','cat9_O'],axis=1)

In [ ]:
test=pd.concat([test,pd.get_dummies(test.cat0,prefix='cat0')],axis=1)
test=test.drop(['cat0','cat0_B'],axis=1)
test=pd.concat([test,pd.get_dummies(test.cat1,prefix='cat1')],axis=1)
test=test.drop(['cat1','cat1_B'],axis=1)
test=pd.concat([test,pd.get_dummies(test.cat2,prefix='cat2')],axis=1)
test=test.drop(['cat2','cat2_B'],axis=1)
test=pd.concat([test,pd.get_dummies(test.cat3,prefix='cat3')],axis=1)
test=test.drop(['cat3','cat3_D'],axis=1)
test=pd.concat([test,pd.get_dummies(test.cat4,prefix='cat4')],axis=1)
test=test.drop(['cat4','cat4_D'],axis=1)
test=pd.concat([test,pd.get_dummies(test.cat5,prefix='cat5')],axis=1)
test=test.drop(['cat5','cat5_D'],axis=1)
test=pd.concat([test,pd.get_dummies(test.cat6,prefix='cat6')],axis=1)
#df_test=df_test.drop(['cat6'],axis=1)
test=test.drop(['cat6','cat6_I'],axis=1)
test=pd.concat([test,pd.get_dummies(test.cat7,prefix='cat7')],axis=1)
test=test.drop(['cat7','cat7_I'],axis=1)
test=pd.concat([test,pd.get_dummies(test.cat8,prefix='cat8')],axis=1)
test=test.drop(['cat8','cat8_G'],axis=1)
test=pd.concat([test,pd.get_dummies(test.cat9,prefix='cat9')],axis=1)
test=test.drop(['cat9','cat9_O'],axis=1)

In [ ]:
target = train['target'].values.reshape(-1, 1)
columns = test.columns[1:]

In [ ]:
test.columns

In [ ]:
columns

In [ ]:
len(columns)

In [ ]:
train_oof = np.zeros((300000,))
test_preds = 0
train_oof.shape

In [ ]:
class RegressorModule(nn.Module):
    def __init__(
            self,
            num_units=100,
            nonlin=F.relu,
    ):
        super(RegressorModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(len(columns), num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 20)
        self.output = nn.Linear(20, 1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = F.relu(self.dense1(X))
        X = self.output(X)
        return X

In [ ]:
NUM_EPOCHS = 15 
NUM_FOLDS = 5
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=0)

for f, (train_ind, val_ind) in tqdm(enumerate(kf.split(train, target))):
        #print(f'Fold {f}')
        train_df, val_df = train.iloc[train_ind][columns].values.astype(np.float32), train.iloc[val_ind][columns].values.astype(np.float32)
        train_target, val_target = target[train_ind].astype(np.float32), target[val_ind].astype(np.float32)
        
        model =  NeuralNetRegressor(RegressorModule, lr=0.01, criterion=nn.MSELoss, max_epochs=NUM_EPOCHS, optimizer=optim.Adam, device='cuda')
        model.fit(train_df, train_target)
        temp_oof = model.predict(val_df)
        temp_test = model.predict(test[columns].values.astype(np.float32))

        train_oof[val_ind] = temp_oof.flatten()
        test_preds += temp_test.flatten()/NUM_FOLDS
        print("Fold error:", mean_squared_error(temp_oof, val_target, squared=False))

In [ ]:
print("Total error:", mean_squared_error(train_oof, target.flatten(), squared=False))

In [ ]:
sub['target'] = test_preds
sub.to_csv('submission.csv', index=False)